In [151]:
datetime.date.today()

datetime.date(2018, 11, 23)

In [157]:
cs = None
assert isinstance(cs, (ClothesSet, type(None)))

In [160]:
import sys; sys.path.insert(0, '..')
import pandas as pd
import datetime

from objects.clothes import ClothesSet, ZONES, CLOTHING_ITEMS, ZONES_NAMES_RU
from objects.weather_forecast import ForecastFrame
from constants.global_constants import USERS_PATH


TABLE_COLUMNS = [*['clothing_item_{}'.format(clothing_item_id) for clothing_item_id in CLOTHING_ITEMS],
                 *['score_{}'.format(zone_name) for zone_name in ZONES_NAMES_RU],
                 *['forecast_frame_{}'.format(key) for key in ForecastFrame().__dict__.keys()],
                 'date']

class HistoryRecord:
    def __init__(self, clothes_set=None, zones_scores=None, forecast_frame=None, date=None):
        assert clothes_set is None or     isinstance(clothes_set, ClothesSet)
        assert forecast_frame is None or  isinstance(forecast_frame, ForecastFrame)
        assert date is None or            isinstance(date, datetime.date)
        if zones_scores is not None:
            assert len(zones_scores) == len(ZONES)
            assert min(zones_scores) in range(-5, 6) 
            assert max(zones_scores) in range(-5, 6)
        
        self.clothes_set = clothes_set
        self.zones_scores = zones_scores
        self.forecast_frame = forecast_frame
        self.date = date
    
    def get_clothes_set(self):
        return self.clothes_set
    
    def get_zones_scores(self):
        return self.zones_scores
    
    def get_forecast_frame(self):
        return self.forecast_frame
    
    def get_date(self):
        return self.date

    @staticmethod
    def from_dict(params_dict):
        assert set(params_dict.keys()) == set(TABLE_COLUMNS)
        def extract_prefixed_dict(dicti, prefix):
            return {key[len(prefix):] : val for key, val in dicti.items() if str(key).startswith(prefix)}
        forecast_frame_items = extract_prefixed_dict(params_dict, prefix='forecast_frame_')
        clothing_items = extract_prefixed_dict(params_dict, prefix='clothing_item_')
        scores_items = extract_prefixed_dict(params_dict, prefix='score_')
        clothes_set = ClothesSet()
        for clothing_item_id, wearing_degree_id in clothing_items.items():
            clothes_set.wear(int(clothing_item_id), wearing_degree_id=int(wearing_degree_id))
        forecast_frame = ForecastFrame(**forecast_frame_items)
        zones_scores = [0]*len(ZONES)
        for key, value in scores_items.items():
            zones_scores[ZONES_NAMES_RU.index(key)] = value
        date = eval(params_dict['date'])
        return HistoryRecord(clothes_set=clothes_set, zones_scores=zones_scores, 
                             forecast_frame=forecast_frame, date=date)
    
    def to_dict(self):
        params_dict = dict()
        params_dict = {key : None for key in TABLE_COLUMNS}
        params_dict.update({'clothing_item_{}'.format(clothing_item_id) : self.clothes_set[clothing_item_id]
                            for clothing_item_id in CLOTHING_ITEMS})
        params_dict.update({'forecast_frame_{}'.format(key) : value 
                            for key, value in self.forecast_frame.__dict__.items()})
        params_dict.update({'score_{}'.format(ZONES_NAMES_RU[key]) : self.zones_scores[key] 
                            for key in range(len(ZONES))})
        params_dict.update({'date' : test_date.__repr__()})
        return params_dict



class HistoryTable:
    def __init__(self, history_records):
        ''' accepts a list of (HistoryRecord)s '''
        for history_record in history_records:
            assert isinstance(history_record, HistoryRecord)
        self.history_records = history_records
    
    @staticmethod
    def from_dataframe(history_df):
        pre_history_records = history_df.to_dict(orient='index')
        assert len(pre_history_records) == max(list(pre_history_records.keys())) + 1
        history_records = [pre_history_records[i] for i in range(len(pre_history_records))]
        for ind, pre_history_record in enumerate(history_records):
            history_records[ind] = HistoryRecord.from_dict(pre_history_record)
        return HistoryTable(history_records)

    def to_dataframe(self):
        return pd.DataFrame(list(map(HistoryRecord.to_dict, self.history_records)))
        

class UserHistoryTable:
    def __init__(self, user_id):
        self.load(user_id)
    
#     # maybe enable the autosave at destruction
#     def __del__(self):
#         self.save()

    def _get_folder(self):
        return os.path.join(USERS_PATH, self.user_id)
    
    def _get_filename(self):
        return os.path.join(self._get_folder(), 'history.csv')
    
    def get_dataframe_representation(self):
        """
        Returns the inner representation as a dataframe. 
        Changing it will not affect the state of the table 
        """
        return self.history_table.to_dataframe()
    
    def get_history_table(self):
        return self.history_table

    def load(self, user_id):
        self.user_id = user_id
        os.makedirs(self._get_folder(), exist_ok=True)
        
        if os.path.exists(self._get_filename()):
            history_df = pd.read_csv(self._get_filename(), index_col=0)
        else:
            history_df = pd.DataFrame(columns=TABLE_COLUMNS)
        self.history_table = HistoryTable.from_dataframe(history_df)
    
    def save(self):
        os.makedirs(self._get_folder(), exist_ok=True)
        history_df = self.history_table.to_dataframe()
        history_df.to_csv(path_or_buf=self._get_filename())

class UserPendingHistoryTable(UserHistoryTable):
    def __init__(self, user_id):
        super(UserHistoryTable, self).__init__(user_id)
        
    def _get_filename(self):
        return os.path.join(self._get_folder(), 'pending_history.csv')

In [113]:
test_user_id = '0'*9

In [114]:
test_clothes_set = ClothesSet([0, 3, 1002, 2006, 3003])
test_clothes_set.wear(1001, 2)
test_scores = [-1, 1, 0, 0, -1]
test_forecast_frame = ForecastFrame(temperature=10, feels_like=5, wind_speed=5, humidity=60, pressure=0.9, common_state='ветер')
test_date = datetime.date(2018, 11, 10)

In [115]:
params_dict = {key : None for key in TABLE_COLUMNS}
params_dict.update({'clothing_item_{}'.format(clothing_item_id) : test_clothes_set[clothing_item_id]
                    for clothing_item_id in CLOTHING_ITEMS})
params_dict.update({'forecast_frame_{}'.format(key) : value 
                    for key, value in test_forecast_frame.__dict__.items()})
params_dict.update({'score_{}'.format(ZONES_NAMES_RU[key]) : test_scores[key] 
                    for key in range(len(ZONES))})
params_dict.update({'date' : test_date.__repr__()})
#params_dict

In [88]:
HistoryRecord.from_dict(params_dict).to_dict() == params_dict

True

In [116]:
test_df = pd.DataFrame( [history_record.to_dict(), history_record.to_dict()], columns=TABLE_COLUMNS )
test_df

,clothing_item_0,clothing_item_1,clothing_item_2,clothing_item_3,clothing_item_4,clothing_item_5,clothing_item_6,clothing_item_1000,clothing_item_1001,clothing_item_1002,...,score_тело,score_голова,score_руки,forecast_frame_common_state,forecast_frame_pressure,forecast_frame_humidity,forecast_frame_wind_speed,forecast_frame_feels_like,forecast_frame_temperature,date
0,3,0,0,3,0,0,0,0,2,3,...,0,0,-1,ветер,0.9,60,5,5,10,"datetime.date(2018, 11, 10)"
1,3,0,0,3,0,0,0,0,2,3,...,0,0,-1,ветер,0.9,60,5,5,10,"datetime.date(2018, 11, 10)"


In [117]:
pre_table = test_df.to_dict(orient='index')
assert len(pre_table) == max(list(pre_table.keys())) + 1
table = [pre_table[i] for i in range(len(pre_table))]
#table

In [118]:
restored_df = pd.DataFrame(table)

In [119]:
restored_df

,clothing_item_0,clothing_item_1,clothing_item_1000,clothing_item_1001,clothing_item_1002,clothing_item_1003,clothing_item_1004,clothing_item_1005,clothing_item_2,clothing_item_2000,...,forecast_frame_feels_like,forecast_frame_humidity,forecast_frame_pressure,forecast_frame_temperature,forecast_frame_wind_speed,score_голова,score_ноги,score_руки,score_ступни,score_тело
0,3,0,0,2,3,0,0,0,0,0,...,5,60,0.9,10,5,0,1,-1,-1,0
1,3,0,0,2,3,0,0,0,0,0,...,5,60,0.9,10,5,0,1,-1,-1,0


In [124]:
history_table = HistoryTable.from_dataframe(test_df)

In [125]:
history_table.to_dataframe()

,clothing_item_0,clothing_item_1,clothing_item_1000,clothing_item_1001,clothing_item_1002,clothing_item_1003,clothing_item_1004,clothing_item_1005,clothing_item_2,clothing_item_2000,...,forecast_frame_feels_like,forecast_frame_humidity,forecast_frame_pressure,forecast_frame_temperature,forecast_frame_wind_speed,score_голова,score_ноги,score_руки,score_ступни,score_тело
0,3,0,0,2,3,0,0,0,0,0,...,5,60,0.9,10,5,0,1,-1,-1,0
1,3,0,0,2,3,0,0,0,0,0,...,5,60,0.9,10,5,0,1,-1,-1,0


In [128]:
history_table.history_records[1].clothes_set[0] += 1

In [129]:
history_table.to_dataframe()

,clothing_item_0,clothing_item_1,clothing_item_1000,clothing_item_1001,clothing_item_1002,clothing_item_1003,clothing_item_1004,clothing_item_1005,clothing_item_2,clothing_item_2000,...,forecast_frame_feels_like,forecast_frame_humidity,forecast_frame_pressure,forecast_frame_temperature,forecast_frame_wind_speed,score_голова,score_ноги,score_руки,score_ступни,score_тело
0,3,0,0,2,3,0,0,0,0,0,...,5,60,0.9,10,5,0,1,-1,-1,0
1,4,0,0,2,3,0,0,0,0,0,...,5,60,0.9,10,5,0,1,-1,-1,0


In [132]:
df_to_test_write = history_table.to_dataframe()

In [134]:
!pwd

/home/igor/HSE/FirstSemester/NIS/Hometasks/clothessuggestingbot/experiments


In [137]:
df_to_test_write.to_csv(path_or_buf='./test.csv')

In [141]:
df_read = pd.read_csv('./test.csv', index_col=0)

In [142]:
df_read == df_to_test_write

,clothing_item_0,clothing_item_1,clothing_item_1000,clothing_item_1001,clothing_item_1002,clothing_item_1003,clothing_item_1004,clothing_item_1005,clothing_item_2,clothing_item_2000,...,forecast_frame_feels_like,forecast_frame_humidity,forecast_frame_pressure,forecast_frame_temperature,forecast_frame_wind_speed,score_голова,score_ноги,score_руки,score_ступни,score_тело
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [143]:
df_read

,clothing_item_0,clothing_item_1,clothing_item_1000,clothing_item_1001,clothing_item_1002,clothing_item_1003,clothing_item_1004,clothing_item_1005,clothing_item_2,clothing_item_2000,...,forecast_frame_feels_like,forecast_frame_humidity,forecast_frame_pressure,forecast_frame_temperature,forecast_frame_wind_speed,score_голова,score_ноги,score_руки,score_ступни,score_тело
0,3,0,0,2,3,0,0,0,0,0,...,5,60,0.9,10,5,0,1,-1,-1,0
1,4,0,0,2,3,0,0,0,0,0,...,5,60,0.9,10,5,0,1,-1,-1,0
